In [2]:
from langchain_community.document_loaders import PyPDFDirectoryLoader

In [3]:
loader=PyPDFDirectoryLoader("D:\Interview\SQL-Assistant-Experiments\data")
docs = loader.load()
docs

[Document(metadata={'producer': 'PyFPDF 1.7.2 http://pyfpdf.googlecode.com/', 'creator': 'PyPDF', 'creationdate': 'D:20250419123113', 'title': 'Healthcare Schema with SQL', 'source': 'D:\\Interview\\SQL-Assistant-Experiments\\data\\Healthcare_Combined_Schema_final.pdf', 'total_pages': 3, 'page': 0, 'page_label': '1'}, page_content='Table: Doctors\nThe Doctors table stores information about healthcare professionals. Each doctor has a unique ID, along with\npersonal details such as first name, last name, date of birth, gender, contact number, and email address.\nCREATE TABLE Doctors (\n    doctor_id INT PRIMARY KEY,\n    first_name VARCHAR(50),\n    last_name VARCHAR(50),\n    dob DATE,\n    gender VARCHAR(10),\n    phone_number VARCHAR(15),\n    email VARCHAR(100)\n);\nTable: Doctor_Licenses\nThe Doctor_Licenses table records licensing information for each doctor. It includes a unique license ID, the\nassociated doctor ID, license number, issue and expiry dates, and the name of the issu

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100,
    is_separator_regex=False,
)

In [5]:
documents=text_splitter.split_documents(docs)

In [6]:
documents

[Document(metadata={'producer': 'PyFPDF 1.7.2 http://pyfpdf.googlecode.com/', 'creator': 'PyPDF', 'creationdate': 'D:20250419123113', 'title': 'Healthcare Schema with SQL', 'source': 'D:\\Interview\\SQL-Assistant-Experiments\\data\\Healthcare_Combined_Schema_final.pdf', 'total_pages': 3, 'page': 0, 'page_label': '1'}, page_content='Table: Doctors\nThe Doctors table stores information about healthcare professionals. Each doctor has a unique ID, along with\npersonal details such as first name, last name, date of birth, gender, contact number, and email address.\nCREATE TABLE Doctors (\n    doctor_id INT PRIMARY KEY,\n    first_name VARCHAR(50),\n    last_name VARCHAR(50),\n    dob DATE,\n    gender VARCHAR(10),\n    phone_number VARCHAR(15),\n    email VARCHAR(100)\n);\nTable: Doctor_Licenses'),
 Document(metadata={'producer': 'PyFPDF 1.7.2 http://pyfpdf.googlecode.com/', 'creator': 'PyPDF', 'creationdate': 'D:20250419123113', 'title': 'Healthcare Schema with SQL', 'source': 'D:\\Intervi

In [6]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

d:\Interview\inv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ['PINECONE_API_KEY']=os.getenv('PINECONE_API_KEY')

In [ ]:
# from langchain_pinecone import PineconeVectorStore
# Only run once - so i am commenting it as it ran already ad index is created in pinecode vectorstore
# vectorstore=PineconeVectorStore.from_documents(documents=documents, embedding=embeddings, index_name="sql-assistant")

In [23]:
from pinecone import Pinecone
pc=Pinecone()

pc.list_indexes().names()

['hybrif-search-langchain-pinecone', 'sql-assistant']

In [27]:
index_name="sql-assistant"

In [32]:
from langchain_pinecone import PineconeVectorStore

In [30]:
if index_name in pc.list_indexes().names():
    print(f"{index_name} is present")
else:
    print(f"{index_name} is not present")
    vectorstore=PineconeVectorStore.from_documents(documents=documents, embedding=embeddings, index_name=index_name)

sql-assistant is present


In [31]:
vectorstore

In [ ]:
from langchain_community.vectorstores import FAISS
vectorstore=FAISS.from_documents(documents=documents, embedding=embeddings)
vectorstore

In [13]:
retriever=vectorstore.as_retriever()

In [15]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.prompts import ChatPromptTemplate

In [16]:
prompt=ChatPromptTemplate.from_template(
    '''
You are HDBRA (Healthcare Database Request Assistant), an expert backend database engineer specializing in healthcare IT systems.
Your primary responsibility is to process service requests (SRs) from healthcare professionals who need database-level changes or reports that cannot be performed through the user interface.

### Database Context:
You have access to a comprehensive healthcare database with the following schema:
- Doctors (doctor_id, name, gender, email, phone, date_joined, status)
- Doctor_Licenses (license_id, doctor_id, license_number, issue_date, expiry_date, issuing_authority)
- Specializations (specialization_id, name, description)
- Doctor_Specializations (doctor_id, specialization_id, certification_date)
- Hospitals (hospital_id, name, address, city, state, contact_number, email)
- Departments (dept_id, hospital_id, name, floor, wing, head_doctor_id)
- Doctor_Hospital_Assignments (assignment_id, doctor_id, hospital_id, dept_id, start_date, end_date, status)
- Patients (patient_id, name, gender, date_of_birth, address, phone, email, blood_group, registration_date)
- Appointments (appointment_id, patient_id, doctor_id, hospital_id, dept_id, appointment_date, status, notes)

### Your task:
For each service request, provide a professional response with:
1. Action Type: Categorize the request (Update, Correction, Assignment, Report, Data Integrity)
2. Target Table(s): Identify the primary tables affected by the request
3. SQL Solution: Generate the precise SQL query or stored procedure to fulfill the request
4. Explanation: Provide a clear, concise explanation of what the SQL does and any considerations

### Rules:
- Carefully check if the request involves ONLY entities (tables, columns, and record types) that are explicitly listed in the Database Context above.
- Be flexible with common formatting variations (e.g., 'hospital ID' should be understood as referring to 'hospital_id').
- Understand that tables may be related to each other (e.g., patients are connected to hospitals through the Appointments table).
- If the request mentions ANY entity that does not exist in the schema (like 'kids', 'government officer', 'employee', 'insurance'), respond with this exact message: "ERROR: The request mentions entities that do not exist in the database schema. Please verify your request and try again."
- Do not try to interpret non-existent entities as existing ones (e.g., don't interpret 'kids' as 'doctors' or 'patients').

Example of incorrect request: "Update the government officer number for patient ID 1042 to +91-8885544332"
Correct response: "ERROR: The request mentions entities that do not exist in the database schema. Please verify your request and try again."

Example of incorrect request: "List all kids specialized in Neurology currently assigned to hospital ID 3"
Correct response: "ERROR: The request mentions entities that do not exist in the database schema. Please verify your request and try again."

Example of correct request: "Update the phone number for patient ID 1042 to +91-8885544332"
This is valid because 'phone' is a column in the Patients table.

Example of correct request: "List all doctors specialized in Neurology currently assigned to hospital ID 3"
This is valid because it refers to entities that exist in the schema: 'doctors', 'specialized' (Specializations), and 'hospital_id'.

Example of correct request: "Get the number of patients registered in hospital ID 5 last month"
This is valid because it refers to 'hospital_id' in the Hospitals table and 'registration_date' in the Patients table, which can be joined through the Appointments table.

Always prioritize data integrity and follow healthcare data handling best practices. For updates to critical fields, include appropriate WHERE clauses to ensure precise targeting.

{context}
### Service Request:
{input}

### Response:
[If the request mentions ANY entity that does not exist in the database schema, respond ONLY with the error message: "ERROR: The request mentions entities that do not exist in the database schema. Please verify your request and try again."]

[For all valid requests that only reference entities in the schema (with allowance for formatting variations like 'hospital ID' for 'hospital_id'), provide the following information:]
Action Type:
Target Table(s):
SQL Solution:
sql
-- SQL query here

Explanation:
'''
)

In [ ]:
# prompt=ChatPromptTemplate.from_template(
#     '''
# You are HDBRA (Healthcare Database Request Assistant), an expert backend database engineer specializing in healthcare IT systems.
# Your primary responsibility is to process service requests (SRs) from healthcare professionals who need database-level changes or reports that cannot be performed through the user interface.

# ### Database Context:
# You have access to a comprehensive healthcare database with the following schema:
# - Doctors (doctor_id, name, gender, email, phone, date_joined, status)
# - Doctor_Licenses (license_id, doctor_id, license_number, issue_date, expiry_date, issuing_authority)
# - Specializations (specialization_id, name, description)
# - Doctor_Specializations (doctor_id, specialization_id, certification_date)
# - Hospitals (hospital_id, name, address, city, state, contact_number, email)
# - Departments (dept_id, hospital_id, name, floor, wing, head_doctor_id)
# - Doctor_Hospital_Assignments (assignment_id, doctor_id, hospital_id, dept_id, start_date, end_date, status)
# - Patients (patient_id, name, gender, date_of_birth, address, phone, email, blood_group, registration_date)
# - Appointments (appointment_id, patient_id, doctor_id, hospital_id, dept_id, appointment_date, status, notes)

# ### Your task:
# For each service request, provide a professional response with:
# 1. Action Type: Categorize the request (Update, Correction, Assignment, Report, Data Integrity)
# 2. Target Table(s): Identify the primary tables affected by the request
# 3. SQL Solution: Generate the precise SQL query or stored procedure to fulfill the request
# 4. Explanation: Provide a clear, concise explanation of what the SQL does and any considerations

# ### Rules:
# - Only respond to service requests that exactly matches tables and columns listed in Database Context.
# - If the request refers to any unknown table or column, then dont reply.
# - Do NOT guess or assume any mappings. Only respond if the input clearly matches the schema.


# Always prioritize data integrity and follow healthcare data handling best practices. For updates to critical fields, include appropriate WHERE clauses to ensure precise targeting.

# {context}
# ### Service Request:
# {input}

# ### Response:
# Action Type: 
# Target Table(s): 
# SQL Solution:
# sql
# -- SQL query here

# Explanation: 
# '''
# )

In [17]:
from langchain_google_genai import ChatGoogleGenerativeAI
import os
from dotenv import load_dotenv
load_dotenv()
llm=ChatGoogleGenerativeAI(model="gemini-2.0-flash",api_key=os.getenv("GEMINI_API_KEY"))

In [18]:
document_chain=create_stuff_documents_chain(llm=llm,prompt=prompt)
retrieval_chain=create_retrieval_chain(retriever,document_chain)

In [ ]:
response=retrieval_chain.invoke({
"input":"Update the phone number for patient ID 1042 to +91-8885544332"
})
response['answer']

'ERROR: The request mentions entities that do not exist in the database schema. Please verify your request and try again.'

In [ ]:
# response=retrieval_chain.invoke(
#     {"input":"Update all phone numbers starting with 99999 to 88888 in the Patients table. List all departments on the 3rd floor in hospital ID 2. Show all appointments scheduled for next week across all hospitals"}
# )

In [ ]:
# response['answer']

"```sql\nUPDATE Patients SET phone_number = REPLACE(phone_number, '99999', '88888') WHERE phone_number LIKE '99999%';\nSELECT department_name FROM Departments WHERE floor_number = 3 AND hospital_id = 2;\nSELECT * FROM Appointments WHERE appointment_date BETWEEN TRUNC(SYSDATE + 7) AND TRUNC(SYSDATE + 14);\n```\n\n*Explanation:*\n```\nThe first query updates the phone numbers in the Patients table, replacing the '99999' prefix with '88888' where applicable.\nThe second query retrieves the names of departments located on the 3rd floor of the hospital with hospital_id 2.\nThe third query retrieves all appointments scheduled for the next 7 days.\n```\n\n*Action Type:* Update, Report\n*Target Table(s):* Patients, Departments, Appointments\n*SQL Query:*\n```sql\nUPDATE Patients SET phone_number = REPLACE(phone_number, '99999', '88888') WHERE phone_number LIKE '99999%';\nSELECT department_name FROM Departments WHERE floor_number = 3 AND hospital_id = 2;\nSELECT * FROM Appointments WHERE appoin